In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn

In [ ]:
from transformers import BertTokenizer
#the model constructed previously
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/modeller/output_bert_base', truncation=True)  

for the Kogan Levin collection

In [ ]:
import pandas as pd
#dataframe with text year volatility values
df = pd.read_csv("/content/drive/MyDrive/my_data.csv")
df.columns = ['Text', 'year', 'real','before']

for the data after 2006

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/later2007data.csv")
df.columns = ['Text', 'year', 'real','before','docname']

# for the collection of Kogan and Levin

In [ ]:
docs = "/content/drive/MyDrive/docs2/"
vols = "/content/drive/MyDrive/vols/vols"

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from csv import writer
import glob

def embed_bert(model_name,tokenizer_name):#(df,model_name,tokenizer_name):
  tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
  model = BertModel.from_pretrained(model_name)
  lengths =[]
  for i in range(1996,2007):
    print("year",i)
    path = docs+str(i)+".mda"
    # All files 
    for file in glob.glob(path+"/*.mda"):
      f = open(file,encoding="latin")
      text= f.read()
      with open('res_data.csv', 'a') as f_object:
        # Pass this file object to csv.writer()
        # and get a writer object
        writer_object = writer(f_object)

        # Pass the list as an argument into
        # the writerow()
        c = list(embedding(text,tokenizer,model))
        List = [file,c[0],c[1]]
        writer_object.writerow(List)

        #Close the file object
        f_object.close()
      f.close()


# for the data after 2006

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from csv import writer
import glob
import csv
from csv import writer
def isNaN(string):
    return string != string
def embed_bert(model_name,tokenizer_name):#(df,model_name,tokenizer_name):
  tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
  model = BertModel.from_pretrained(model_name)
  #f = open(file,'a')
  #df2 = pd.DataFrame([list(embedding(df["Text"][0],tokenizer,model))],columns=['mean','std'])
  for i in range(24175,len(df)):
    with open('/content/drive/MyDrive/result_data.csv', 'a') as f_object:
      print('row_number:',i)
      # Pass this file object to csv.writer()
      # and get a writer object
      writer_object = writer(f_object)
      text = df.iloc[i]['Text']
      if not isNaN(text):
        real = df.iloc[i]['real']
        year = df.iloc[i]['year']
        before = df.iloc[i]['before']
        name = df.iloc[i]['docname']
        # Pass the list as an argument into
        # the writerow()
        c,d = embedding(text,tokenizer,model)
        List = [name,c,d,real,before,year] #docname v1 v2 real before name
        writer_object.writerow(List)

        #Close the file object
        f_object.close()
      else:
        f_object.close()
  #f.write(str(embedding(df["Text"][i],tokenizer,model)))
  #df2= df['Text'].apply(lambda x: embedding(x,tokenizer,model))
  #f.close()
  #return df2

# embedding extarction files

In [ ]:
def into_the_model(model,my_input_ids,my_attention_mask):
  my_input_ids = torch.stack(tuple(torch.from_numpy(my_input_ids)))
  my_attention_mask = torch.stack(tuple(torch.from_numpy(my_attention_mask)))
  #my_token_type_ids = torch.stack(tuple(torch.from_numpy(my_token_type_ids)))

  input_dict = {
      'input_ids': my_input_ids.long(),
      #'token_type_ids':my_token_type_ids.int(),
      'attention_mask': my_attention_mask.int()
  }
  outputs = model(**input_dict)
  last_hidden_states = outputs.last_hidden_state
  return last_hidden_states #torch.Tensor type
def embedding(text, tokenizer,model):
  inputs = tokenizer.encode_plus(text,'padding=True','truncation=True', return_tensors="pt")
  if inputs['input_ids'].size()[1]>510:
    input_ids=inputs['input_ids'][0].split(510)
    #token_type_ids =inputs['token_type_ids'][0].split(510)
    attention_mask=inputs['attention_mask'][0].split(510)
    length = len(input_ids)
    count = 0
    #lastest = torch.Tensor()
    lastest = np.empty([0])
    my_input_ids =np.empty([0])
    print(len(my_input_ids))
    my_attention_mask=np.empty([0])
    #my_token_type_ids=np.empty([0])
    while count<length:
      pad_len = 512 - input_ids[count].shape[0]
      # check if tensor length satisfies required chunk size
      if pad_len > 0:
          # if padding length is more than 0, we must add padding
          if count>0:
            my_input_ids=np.vstack((my_input_ids, torch.cat([
                input_ids[count], torch.Tensor([0] * pad_len)] # torch.Tensor([0] * pad_len)
            ).detach().numpy()))
            print(len(my_input_ids))
            my_attention_mask=np.vstack((my_attention_mask, torch.cat([
                attention_mask[count], torch.Tensor([0] * pad_len)]
            ).detach().numpy() ) )
            #my_token_type_ids=np.vstack((my_token_type_ids,torch.cat([
            #    token_type_ids[count], torch.Tensor([1] * pad_len)]).detach().numpy()
            #))
          else:
            my_input_ids=torch.cat([
                input_ids[count], torch.Tensor([0] * pad_len)] # torch.Tensor([0] * pad_len)
            ).detach().numpy()
            print(len(my_input_ids))
            my_attention_mask= torch.cat([
                attention_mask[count], torch.Tensor([0] * pad_len)]
            ).detach().numpy()
            #my_token_type_ids=torch.cat([
            #    token_type_ids[count], torch.Tensor([1] * pad_len)]).detach().numpy()

      count+=1
    if len(my_input_ids)>30:
      lastest = np.empty([0])
      b = int(len(my_input_ids)/30)
      parts_input_ids = np.split(my_input_ids[:30*b],b)
      parts_attention_mask = np.split(my_attention_mask[:30*b],b)
      if len(my_input_ids)%30!=0:
        parts_input_ids.append(my_input_ids[30*b:])
        parts_attention_mask.append(my_attention_mask[30*b:])
        b = b+1
      for i in range(b):
        print(i)
        if i>0:
          lastest =np.vstack((lastest ,into_the_model(model,parts_input_ids[i],parts_attention_mask[i]).detach().numpy()  ))
        else: 
          lastest = into_the_model(model,parts_input_ids[i],parts_attention_mask[i]).detach().numpy()
    else:
      lastest =into_the_model(model,my_input_ids,my_attention_mask)
    lastest = lastest.reshape(lastest.shape[0]*lastest.shape[1],768)
  else:
    outputs = model(**inputs)
    lastest = outputs.last_hidden_state.detach().numpy()
  mean = np.mean(lastest, axis =0)
  std = np.std(lastest, axis =0)
  return mean, std
  #return torch.mean(lastest[0], 0).detach().numpy(),torch.std(lastest[0], 0).detach().numpy()


In [ ]:
embed_bert('/content/drive/MyDrive/modeller/output_bert_base','/content/drive/MyDrive/modeller/output_bert_base') 
# for the bert embedings, embed_bert function was 
# embed_bert('bert-base-uncased','bert-base-uncased')

# modifications in functions to find closer words

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))
stop_words.append('could')
stop_words.append('although')
#stop_words.append('code')
stop_words.append('despite')
stop_words.append('within')
stop_words.append('also')
stop_words.append('pad')
stop_words.append('cls')
def seven_up(lastest,input_ids,vector,tokenizer):
  tuple_list = []
  ret_list =[]
  for index in range(len(lastest)):
    temp = lastest[index][:] - vector
    sum_sq = np.dot(temp.T, temp)
    tuple_list.append((input_ids[index],sum_sq))
  tuple_list.sort(key=lambda x:x[1])
  words = [tokenizer.decode(x[0]) for x in tuple_list]
  words = [x.lower() for x in words]
  words = [x for x in words if x!="[pad]" and x not in string.punctuation and x not in stop_words and not x.isnumeric()]
  print("there are ",len(words))
  return words

In [ ]:
def into_the_model(model,my_input_ids,my_attention_mask):
  my_input_ids = torch.stack(tuple(torch.from_numpy(my_input_ids)))
  my_attention_mask = torch.stack(tuple(torch.from_numpy(my_attention_mask)))
  #my_token_type_ids = torch.stack(tuple(torch.from_numpy(my_token_type_ids)))

  input_dict = {
      'input_ids': my_input_ids.long(),
      #'token_type_ids':my_token_type_ids.int(),
      'attention_mask': my_attention_mask.int()
  }
  outputs = model(**input_dict)
  last_hidden_states = outputs.last_hidden_state
  return last_hidden_states #torch.Tensor type
def embedding(text, tokenizer,model):
  ##text = text.lower()
  ##text = text.translate()
  ##words = [x for x in words if x!="[pad]" and x not in string.punctuation and x not in stop_words and not x.isnumeric()]
  inputs = tokenizer.encode_plus(text,'padding=True','truncation=True', return_tensors="pt")
  if inputs['input_ids'].size()[1]>510:
    input_ids=inputs['input_ids'][0].split(510)
    #token_type_ids =inputs['token_type_ids'][0].split(510)
    attention_mask=inputs['attention_mask'][0].split(510)
    length = len(input_ids)
    count = 0
    #lastest = torch.Tensor()
    lastest = np.empty([0])
    our_input_ids = np.empty([0])
    my_input_ids =np.empty([0])
    print(len(my_input_ids))
    my_attention_mask=np.empty([0])
    #my_token_type_ids=np.empty([0])
    while count<length:
      pad_len = 512 - input_ids[count].shape[0]
      # check if tensor length satisfies required chunk size
      if pad_len > 0:
          # if padding length is more than 0, we must add padding
          if count>0:
            my_input_ids=np.vstack((my_input_ids, torch.cat([
                input_ids[count], torch.Tensor([0] * pad_len)] # torch.Tensor([0] * pad_len)
            ).detach().numpy()))
            #print(len(my_input_ids))
            our_input_ids =np.vstack((our_input_ids,torch.cat([
                input_ids[count], torch.Tensor([0] * pad_len)]).detach().numpy().reshape( 512,1)) )
            #print(len(our_input_ids))
            my_attention_mask=np.vstack((my_attention_mask, torch.cat([
                attention_mask[count], torch.Tensor([0] * pad_len)]
            ).detach().numpy() ) )
            #my_token_type_ids=np.vstack((my_token_type_ids,torch.cat([
            #    token_type_ids[count], torch.Tensor([1] * pad_len)]).detach().numpy()
            #))
          else:
            my_input_ids=torch.cat([
                input_ids[count], torch.Tensor([0] * pad_len)] # torch.Tensor([0] * pad_len)
            ).detach().numpy()
            our_input_ids =my_input_ids.reshape(len(my_input_ids),1)
            #print(len(my_input_ids))
            my_attention_mask= torch.cat([
                attention_mask[count], torch.Tensor([0] * pad_len)]
            ).detach().numpy()
            #my_token_type_ids=torch.cat([
            #    token_type_ids[count], torch.Tensor([1] * pad_len)]).detach().numpy()

      count+=1
    if len(my_input_ids)>30:
      lastest = np.empty([0])
      b = int(len(my_input_ids)/30)
      parts_input_ids = np.split(my_input_ids[:30*b],b)
      parts_attention_mask = np.split(my_attention_mask[:30*b],b)
      if len(my_input_ids)%30!=0:
        parts_input_ids.append(my_input_ids[30*b:])
        parts_attention_mask.append(my_attention_mask[30*b:])
        b = b+1
      for i in range(b):
        #print(i)
        if i>0:
          lastest =np.vstack((lastest ,into_the_model(model,parts_input_ids[i],parts_attention_mask[i]).detach().numpy()  ))
          
        else: 
          lastest = into_the_model(model,parts_input_ids[i],parts_attention_mask[i]).detach().numpy()
          
    else:
      lastest =into_the_model(model,my_input_ids,my_attention_mask).detach().numpy()
      
    lastest = lastest.reshape(lastest.shape[0]*lastest.shape[1],768)
  else:
    outputs = model(**inputs)
    lastest = outputs.last_hidden_state.detach().numpy()
  #my_input_ids = my_input_ids.reshape(my_input_ids.shape[1] *my_input_ids.shape[0],1)
  print("lastest shape",lastest.shape,"input_ids shape ", our_input_ids.shape)
  print(len(lastest[0]))
  mean = np.mean(lastest, axis =0)
  std = np.std(lastest, axis =0)
  wm = seven_up(lastest,our_input_ids,mean,tokenizer)
  ws = seven_up(lastest,our_input_ids,std,tokenizer)
  #print(wm)
  print(len(wm))
  #return 0
  return wm[-100:],ws[-100:]